In [1]:
import cv2
import numpy as np
from importnb import Notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy.fft as fft
import numpy.matlib
from skimage.transform import rotate
import sys

In [2]:
with Notebook(): 
        import utils
        import pinkNoise

In [3]:
def blurLegend(directions, magnitude):
    norm = mpl.colors.Normalize(0, np.pi)
    plt.imshow(norm(directions))
    plt.show()

In [4]:
def makePatchKernel(img, r_min=5, r_max=31, inc_th=15):
    nr = r_max - r_min
    nt = int(180/inc_th)
    ind = []
    kernels = []
    ker_centre = (r_max*r_max)//2
#     kernels = np.zeros(nr*nt, r_max, img.shape[0]*img.shape[1])
    for r in range(r_min, r_max+1, 2):
        for theta in range(0, 180, inc_th):
            ind.append((r,theta))
            temp = np.zeros((r_max*r_max))
            ker = utils.blurKernel(r, theta).ravel()
#             print(temp[ker_centre-ker.shape[0]//2:ker_centre-ker.shape[0]//2].shape)
            temp[ker_centre-ker.shape[0]//2:ker_centre+ker.shape[0]//2+1] = ker
            kernels.append(temp)
#             print("\n\n", r, theta)
#             print(ker)
    return np.array(kernels), np.array(ind)

In [5]:
def findPatchKernel(img, kernels, ind, r_min=5, r_max=31, inc_ang=15):
    mse = float('inf')
    opt_r, opt_theta = 1, 0
    if r_min % 2==0:
        r_min += 1
    if r_max % 2==0:
        r_max += 1
#     print(img.shape)
# #     kernels, ind = makePatchKernel(img, r_min, r_max, inc_ang)
#     pad_img = np.pad(img, ((r_max//2, r_max//2), (r_max//2, r_max//2)), 'constant', constant_values=(0, 0))
#     col_img = utils.im2col(pad_img, (r_max, r_max))
# #     out = kernels@col_img - img.ravel()
#     out = np.sum(np.power(kernels@col_img - img.ravel(), 2), axis=1)
#     print(out.shape)
    for r in range(r_min, r_max, 2):
        for theta in range(0, 180, inc_ang):
            blur = utils.linearBlur(img, r, theta)
            err = utils.MSELoss(img, blur)
            if err < mse:
                mse = err
        opt_r, opt_theta = r, theta

    return ind[out.argmax()]

In [6]:
def getImageBlurEstimate(img, patch_size = 31):
    kernels, ind = makePatchKernel(img)    
    blurField = np.zeros(img.shape)
    magnitude = np.zeros(img.shape)
    pd = patch_size // 2
    img = np.pad(img, ((pd, pd), (pd, pd)), 'reflect')
    from tqdm import tqdm_notebook as tqdm
    
    pd+=1
    for i in tqdm(range(pd, img.shape[0]-pd)):
        for j in range(pd, img.shape[1]-pd):
            print(i, j)
#             print(i-pd, j-pd, end = '\r')
#             print(img[i-pd:i+pd+1,j-pd:j+pd+1].shape)
            magnitude[i-pd,j-pd], blurField[i-pd,j-pd] = findPatchKernel(img[i-pd:i+pd+1,j-pd:j+pd+1], kernels, ind)
#             magnitude[i-pd,j-pd], blurField[i-pd,j-pd] = findPatchKernel(img[i-pd:i+pd,j-pd:j+pd], 7, 8, 15)

    return magnitude, blurField

# TESTING

In [7]:
# img = utils.loadImage('./resources/blu.png', 'gray')
# img = utils.loadImage('./resources/building.jpg', 'gray')
# blurred_img = utils.linearBlur(img, 10, 45)
# noise, noised_img = pinkNoise.addNoise(blurred_img, exponent = -1)

# _, plots = plt.subplots(1,2,figsize=(20,20))
# plots[0].imshow(noise, cmap = 'gray')
# plots[1].imshow(noised_img, cmap = 'gray')
# plt.show()



In [8]:
# magnitude, directions = getImageBlurEstimate(img[200:235,200:235])
# # magnitude, directions = findPatchKernel(img[100:100,200:350])
# print(magnitude, directions)

In [9]:
# blurLegend(directions, magnitude)
# np.set_printoptions(threshold=sys.maxsize)
# print((directions==45).sum()/(150*150))